In [ ]:
import numpy
%matplotlib inline
from matplotlib import pyplot
pyplot.rc('font', family='serif', size=5)

In [ ]:
import sys
sys.path.append('../scripts/')

# Our helper
from plot_helper import *

In the previous notebook, we visually explained what are eigenvectors and eigenvalues, and we can use `numpy.linalg.eig()` to solve for the eigenpairs. Take $\,A = \begin{bmatrix} 1 & 0 \\ 1 & 3 \end{bmatrix}$ as an example.

In the previous notebook, we visually explained what are eigenvectors and eigenvalues, and we can use `numpy.linalg.eig()` to solve for the eigenpairs. Take $\,A = \begin{bmatrix} 1 & 2 \\ 2 & 1 \end{bmatrix}$ as an example.

In [ ]:
A = numpy.array([[1,2], [2,1]])
d, C = numpy.linalg.eig(A)
for eigenvalue, eigenvector in zip(d, C.T):
    print(eigenvalue, eigenvector)

$d$ is an array of eigenvalues and each column of $C$ represents a normalized eigenvector of matrix $A$. We can decompose the matrix as $A = C\, D\, C^{-1}$ where $D$ is the diagonal matrix of $d$. Let us draw both eigenvectors along with this linear transformation using `plot_linear_transformation()`, and set `unit_circle=True` to show the unit circle and `unit_vector=False` to hide the unit vectors.

In [ ]:
plot_linear_transformation(A, C[:,0], C[:,1], unit_vector=False, unit_circle=True)

The eigenvectors (in pink and dark blue) of $A$ will not change their directions after the transformation of $A$. Since $A$ is symmetric, the transformed eigenvectors happen to be the semi-axes of the transformed unit circle.

If we look at a general case where $A$ is an asymmetric matrix, say $\,A = \begin{bmatrix} 1 & 0 \\ 1 & 3 \end{bmatrix}$, we will observe that the transformed eigenvectors still keep their original directions, however, they no longer coincide with the semi-axes of the ellipse, as the plot demonstrated below. 

In [ ]:
A = numpy.array([[1,0], [1,3]])
d, C = numpy.linalg.eig(A)
plot_linear_transformation(A, C[:,0], C[:,1], unit_vector=False, unit_circle=True)

Then, how could we find the vectors that correspond to the semi-axes of the ellipse? They are intriguing because they are the most stretched or most shrunk vectors after the transformation. Similar to what we've done before, we calculate the maximum and minimum distance from the origin to the ellipse. The longest vector between them is the semi-major axis, the shortest one is the semi-minor axis.

In [ ]:
alpha = numpy.linspace(0, 2*numpy.pi, 201)
circle = numpy.vstack((numpy.cos(alpha), numpy.sin(alpha)))
ellipse = A @ circle    # 2 by 41 ndarray
distance = numpy.linalg.norm(ellipse, axis=0)
major_id = numpy.argmax(distance)
minor_id = numpy.argmin(distance)
major = ellipse[:, major_id]
minor = ellipse[:, minor_id]
print(major, numpy.linalg.norm(major))
print(minor, numpy.linalg.norm(minor))

Recall that `major` and `minor` are transformed vectors which had a unit length before the transformation. To use our plotting function `plot_linear_transformation` to visualize them, we need to pass in the corresponding `major` and `minor` vectors *before* the transformation. We name them `major_before` and `minor_before`.

In [ ]:
A_inv = numpy.linalg.inv(A)
major_before = A_inv @ major
minor_before = A_inv @ minor

We can finally plot the semi-axes now. Notice that we do not need to plot eigenvectors this time.

In [ ]:
plot_linear_transformation(A, major_before, minor_before,
                           unit_vector=False, unit_circle=True)

We all know that the major and minor axes of an ellipse are orthogonal (perpendicular to each other), however, to our surprise, their corresponding vectors `major_before` and `minor_before`, which land on the original unit circle, appear to be orthogonal as well. We can use inner product `numpy.dot()` to confirm.

In [ ]:
print(major.dot(minor))
print(major_before.dot(minor_before))

If we use $\mathbf{u_1}, \mathbf{u_2}$ to denote the *normalized* `major` and `minor` vectors, use $\mathbf{s_1}, \mathbf{s_2}$ to denote the lengths of `major` and `minor`, and use $\mathbf{v_1}, \mathbf{v_2}$ to denote `major_before` and `minor_before` (they are normalized already since they land on the unit circle), we can formulate the observation above as:

$$
\begin{align*}
  A \mathbf{v_1} = s_1 \mathbf{u_1} \\
  A \mathbf{v_2} = s_2 \mathbf{u_2}
\end{align*}
$$

Stacking two equations together:

$$
  A \begin{bmatrix}
    \mathbf{v_1} & \mathbf{v_2}
    \end{bmatrix}
    =
    \begin{bmatrix}
    \mathbf{u_1} & \mathbf{u_2}
    \end{bmatrix}
    \begin{bmatrix}
    s_1 & 0 \\
    0 & s_2
    \end{bmatrix}  
$$

Using $V$ to denote $\begin{bmatrix} \mathbf{v_1} & \mathbf{v_2} \end{bmatrix}$, $U$ to denote $\begin{bmatrix} \mathbf{u_1} & \mathbf{u_2} \end{bmatrix}$ and $S$ to denote the diagonal scaling matrix, it becomes:

$$
  A\, V = U\, S
$$

then right-multiply by $V^{-1}$ on both sides:

$$
  A = U\, S\, V^{-1}
$$

Since $\mathbf{v_1}, \mathbf{v_2}$ are orthogonal unit vectors, the matrix $V$ is an orthogonal matrix. (Recall what is an orthogonal matrix from the previous notebook) Thus, $V^T = V^{-1}$, the equation can also be written as:

$$
  A = U\, S\, V^{T}
$$

Starting from discovering the semi-axes of the transformed unit circle, we finally arrive at another neat decomposition of the matrix $A$ - singular value decomposition (SVD). The columns of $U$ are called left singular vectors, the columns of $V$ are called right singular vectors, and the diagonal elements of $S$ are singular values.

Let us build these matrices first.

In [ ]:
s1 = numpy.linalg.norm(major)
s2 = numpy.linalg.norm(minor)
S = numpy.diag([s1, s2])
print(S)

In [ ]:
u1 = major / s1
u2 = minor / s2
U = numpy.transpose(numpy.vstack((u1,u2)))
print(U)

In [ ]:
v1 = major_before
v2 = minor_before
VT = numpy.vstack((v1,v2))
print(VT)

We can retrieve matrix $A$ by multiplying them together.

In [ ]:
U @ S @ VT

From the previous notebook, we know that an orthogonal matrix ($U$ and $V$) implies a rotation transformation while a diagonal matrix ($S$) implies a scaling transformation. Let us visualize each component in the SVD. We also draw the unit vectors after each transformation this time.

In [ ]:
plot_linear_transformations(VT, S, U, unit_circle=True)

The matrix $V^T$ first "rotates" the unit circle, then the diagonal matrix $S$ "stretches" the unit circle in both $x$-axis and $y$-axis directions into an ellipse, finally the matrix $U$ "rotates" this ellipse to a new direction. Therefore, the linear transformation of matrix $A$ can be viewed as a combination of rotation and scaling.

## Compute SVD in Python

You can compute the SVD of a matrix using [`numpy.linalg.svd()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.svd.html). It returns a tuple: its first element is a 2D array where each column is a left singular vector, its second element is an array with the singular values, and its third element is a 2D array where each row is a right singular vector.

In [ ]:
U, S, VT = numpy.linalg.svd(A)

Let us loop over each pair of singular values and singular vectors, print them out.

In [ ]:
for u, s, v in zip(U.T, S, VT):
    print(s, u, v)

We can use [`numpy.allclose()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.allclose.html) to confirm that the decomposition is close enough to the matrix $A$.

In [ ]:
A_decomp = U @ numpy.diag(S) @ VT
print(A_decomp)
print(numpy.allclose(A, A_decomp))

As we discussed before, a $3 \times 3$ matrix represents a linear transformation in 3D. We can also apply SVD to it using `numpy` built-in function and visualize each component. The linear transformation is a composition of a rotation, a scaling and another rotation transformation.

In [ ]:
A = numpy.array([[1,2,3], [1,1,1], [-1,1,0]])
U, S, VT = numpy.linalg.svd(A)
plot_3d_linear_transformations(VT, numpy.diag(S), U, unit_sphere=True)

### Rank-deficient matrix

In the second notebook, we introduced the rank of a matrix, the dimension of the vector space spanned by the columns of a matrix. It also describes the number of linearly independent columns of a matrix. When the rank of a matrix does not match the number of columns, the matrix is called *rank-deficient*.

The matrix $N = \begin{bmatrix} 1 & 2 & 7 \\
                                0 & 1 & 3 \\
                                -3 & 1 & 0 \end{bmatrix} $ has rank 2. Plotting this information below, you will find that all the 3d grid lines collapse on a 2d plane.

In [ ]:
N = numpy.array([[1,2,7], [0,1,3], [-3,1,0]])
plot_3d_linear_transformation(N)

With the help of SVD, now we could look at this rank-deficient matrix from a different perspective. Let's plot each component of SVD and turn on `unit_sphere`.

In [ ]:
U, S, VT = numpy.linalg.svd(N)
plot_3d_linear_transformations(VT, numpy.diag(S), U, unit_sphere=True)

The second transformation squishes the unit sphere into an ellipse on the plane $z=0$. Since each singular value describes the scaling ratio along the corresponding axis, what we observed implies that the third singular value, corresponding to $z$-axis, is zero. Let's print out the singular values to verify.

In [ ]:
print(S)

For a $3 \times 3$ matrix, the number of nonzero singular values determines the dimensionality of the unit sphere after the transformation. If the matrix is full-rank, the transformed shape is an ellipsoid; if it has rank 2, the transformed shape is an ellipse; if it has rank 1, the transformed shape is a line segment. We can generalize this property to higher dimensions as follows:

##### Key idea:

> The number of nonzero singular values of square matrix $A$ equals to its rank.

##### Challenge:

> Check the singular values of a rank-1 matrix.

### Low Rank Approximation

In [ ]:
A = numpy.array([[1, 2, 3, 6],
                 [2, 5, 7, 10],
                 [3, 9, 12, 14],
                 [4, 7, 9, 15]])

In [ ]:
U, S, VT = numpy.linalg.svd(A)
print(S)

In [ ]:
numpy.linalg.matrix_rank(A)

In [ ]:
k = 3
A_approx = U[:,:k] @ numpy.diag(S[:k]) @ VT[:k,:]
print(A_approx)

In [ ]:
numpy.linalg.matrix_rank(A_approx)

### Image Compression using SVD

We now understand that each singular value indicate the amount of scaling along its axis. 

In [ ]:
from imageio import imread, imwrite
image = imread('../images/washington-monument.jpg')

In [ ]:
print(type(image))
print(image.shape)
print(image.dtype)

In [ ]:
pyplot.figure(figsize=(2,2))
pyplot.imshow(image, cmap='gray');

In [ ]:
U, S, VT = numpy.linalg.svd(image, full_matrices=False)
print(U.shape, S.shape, VT.shape)

In [ ]:
pyplot.figure(figsize=(3,2))
pyplot.scatter(numpy.arange(len(S)), S, s=0.8)
pyplot.yscale('log')
pyplot.ylabel('singular value');

In [ ]:
from ipywidgets import widgets, fixed
import os

In [ ]:
def approximate(k, u, s, vt, image):
    copy = u[:,:k] @ numpy.diag(s[:k]) @ vt[:k,:]
    copy = copy.round()   # round to integer
    copy = numpy.where(copy>255, 255, copy)
    copy = numpy.where(copy<0, 0, copy)
    copy = copy.astype(numpy.uint8)
    diff = numpy.abs(image - copy)
    fig = pyplot.figure(figsize=(4,2))
    ax1 = pyplot.subplot(121)
    ax1.imshow(copy, cmap='gray')
    ax1.set_title('compressed image'.format(k))
    ax2 = pyplot.subplot(122)
    ax2.imshow(image, cmap='gray')
    ax2.set_title('original image'.format(k))
    
    imwrite('./compressed.jpg', copy)
    orig_size = os.path.getsize('../images/washington-monument.jpg')
    comp_size = os.path.getsize('./compressed.jpg')
    print(comp_size/orig_size)
    fig.suptitle('$k = {}$'.format(k))

In [ ]:
slider = widgets.IntSlider(min=0, max=100, step=5)
widgets.interact(approximate, k=slider, u=fixed(U),
                 s=fixed(S), vt=fixed(VT), image=fixed(image));

### Nonsquare matrix

So far, we have only talked about square matrices. A $2\times 2$ matrix transforms a 2d vector to another 2d vector, and a $3\times 3$ matrix transforms a 3d vector to another 3d vector. What about a nonsquare matrix, for example, a $3 \times 2$ matrix $M =  \begin{bmatrix} 1 & 0 \\ 1 & 1 \\ 2 & 3\end{bmatrix}$ ?

It transforms a 2d vector to a 3d vector.

If we use $\mathbf{i}$ and $\mathbf{j}$ to denote the standard basis vectors. The first column corresponds to the vector where $\mathbf{i}$ lands after the transformation, and the second column is where $\mathbf{j}$ lands:
$$
\mathbf{i} = \begin{bmatrix} 1 \\ 0 \end{bmatrix}  \Rightarrow  \begin{bmatrix} 1 \\ 1 \\ 2 \end{bmatrix} \\
\mathbf{j} = \begin{bmatrix} 0 \\ 1 \end{bmatrix}  \Rightarrow  \begin{bmatrix} 0 \\ 1 \\ 3 \end{bmatrix}
$$

so a vector $\mathbf{x} = \begin{bmatrix} x \\ y \end{bmatrix}$ will land in $ M\mathbf{x} = x \cdot \begin{bmatrix} 1 \\ 1 \\ 2 \end{bmatrix} + y \cdot \begin{bmatrix} 0 \\ 1 \\ 3 \end{bmatrix}$.

##### Key idea:

> An $m\times n$ nonsquare matrice transforms an $n$-dimensional vector to an $m$-dimensional vector.

You can also multiply a nonsquare matrix with another nonsquare matrix, which represents the composition of two linear transformations between dimensions. However, we need to double check the dimensionality of the two matrices.

### SVD for nonsquare matrices

Back to the mathematical representation:

$$
  A \begin{bmatrix}
    \mathbf{v_1} & \mathbf{v_2} & \mathbf{v_3}
    \end{bmatrix}
    =
    \begin{bmatrix}
    \mathbf{u_1} & \mathbf{u_2} & \mathbf{u_3}
    \end{bmatrix}
    \begin{bmatrix}
    s_1 & 0 & 0\\
    0 & s_2 & 0\\
    0 & 0 & s_3
    \end{bmatrix}  
$$

$$
\begin{align*}
  A &=
    \begin{bmatrix}
    \mathbf{u_1} & \mathbf{u_2} & \mathbf{u_3}
    \end{bmatrix}
    \begin{bmatrix}
    s_1 & 0 & 0\\
    0 & s_2 & 0\\
    0 & 0 & s_3
    \end{bmatrix} 
    \begin{bmatrix}
    \mathbf{v_1}^T \\ \mathbf{v_2}^T \\ \mathbf{v_3}^T
    \end{bmatrix} \\
    &=
    \mathbf{u_1} s_1 \mathbf{v_1}^T + \mathbf{u_2} s_2 \mathbf{v_2}^T + \mathbf{u_3} s_3 \mathbf{v_3}^T \\
\end{align*}
$$

In [ ]:
# Execute this cell to load the notebook's style sheet, then ignore it
from IPython.core.display import HTML
css_file = '../style/custom.css'
HTML(open(css_file, "r").read())